## Chlorophyll-a forecasting using LSTM Model



#### Getting Started:
1. Before running the notebook, please make sure to have the following python version and libraries are installed <br>
- python 3.9.12
- pytorch (https://pytorch.org/get-started/locally/)

2. Create an account in Weights and Biases (WANDB) (https://wandb.ai/home). While running the notebook, you maybe prompted to enter the WANDB username

<br>
The requirements.txt file lists the basic libraries require. Running the following cell should install all of them (in case they are not already installed). 

In case, any library is missed here, you would be prompted with an ImportError. In such case, just install it with pip (google -> pip install library_name)

In [1]:
!pip install -r requirements.txt

In [2]:
import random
import pandas as pd
import numpy as np
from tqdm import trange
import os
import datetime
import matplotlib.pyplot as plt
import math

import torch
import torch.nn as nn
from torch import optim

from utils import Utils
from encoder_decoder import seq2seq

import warnings
warnings.filterwarnings('ignore')

wandb: Currently logged in as: rladwig (computational-limnology). Use `wandb login --relogin` to force relogin


## 0. GPU Selection
Check if GPU is available on the machine the notebook is running. If yes, then assign a GPU, else run it on CPU

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


## 1. Parameter setting

#### Specify the wandb project and wandb run
wandb refers to Weights and Biases. Integrating this tool into the notebook will allow it to access the run details and generate train and test curves, among many other information

In [4]:
# wandb project name
wandb_project = "mcl_lstm"

# wandb run name
wandb_run = "conv_run_{}_{}".format(str(datetime.datetime.now().date()), str(datetime.datetime.now().time()))

# Yes if we want wandb to save our python code, else no
save_code = True

#### Specify the input path (where the dataset is stored) and dataset name
Note: For different dataset, the processing/handling can/will be different. In this notebook, FCR (observational) data has been considered. It also has a metadata file that stores the column names and types. 
<br>
For the purpose of the tutorial, the notebook is kept simple, hence, going with FCR data for now

In [5]:
# Input path
path = './'

# Name of the file
file = '../1_trainingData/all_data_lake_modeling_in_time_perRow.csv'

# Name of the metadata file
#../metadata = 'LSTM_dataset_column_key_07OCT22.csv'

#### Specify the Time-series specific parameters

In [6]:
# Lookback window
input_window = 24*3

# horizon window
output_window = 24*3

# stride - While creating samples (lookback window + horizon window = 1 sample) define the amount of stride the sliding window needs to take
stride = 1

# The ratio in which train and test data is split. If it is 0.8, then first 80% of data goes into train and remaining 20% into test
split_ratio = 0.6

#### Specify the model specific parameters

In [7]:
# Types of Model include: LSTM, GRU, RNN
model_type = "LSTM"

# Number of layers in our deep learning model
num_layers = 2

# Hidden cell (RNN/LSTM/GRU) size
hidden_feature_size = 32

# Output size of our encoder_decoder model, i.e. number of target variables
output_size = 50

'''
Model Training parameters
'''
# batch_size during training
batch_size = 32

# Number of epochs we want to train the model for (1 epoch = 1 pass of the complete training data through the model)
epochs = 100

# Learning rate specifies the rate at which we want to update the model parameters after every training pass
learning_rate = 0.001

# Eval freq says how frequently during training do you want to evaluate your model on the validation data (to see its performance on non-training data)
eval_freq = 1 # logic is -> if iteration_num % eval_freq == 0 -> then perform evaluation

# While generating the training batches do we want the generator to shuffle the batches?
batch_shuffle = True

# Dropout is a form of regularization
dropout = 0.0

'''
Learning rate scheduler parameters
'''
max_lr=5e-3
div_factor=100
pct_start=0.05 
anneal_strategy='cos'
final_div_factor=10000.0

'''
Parameters for early stopping
'''
# Set to True if we want Early stopping
early_stop = False

# If there is no improvement for a 'thres' number of epocs stop the training process
thres=5

# Quantifying the improvement. If the validation loss is greater than min_val_loss_so_far + delta for thres number of iterations stop the training
delta=0.5

'''
Other parameters
'''
# Specify the amount of L2 regularization to be applied.
weight_decay=0.0

# Specify the percentage of times we want to enforce teacher forcing
teacher_forcing_ratio = 0.0
training_prediction = 'recursive'

## 2. Data Processing

#### Read the metadata file

In [8]:
depth_steps = 25 * 2 

depth_list = np.array(list(range(1, depth_steps+1))   )*0.5



In [9]:
incoming_temp = ['temp_diff03_{}'.format(x) for x in depth_list]
outgoing_temp = ['temp_conv04_{}'.format(x) for x in depth_list]

dx = pd.read_csv(os.path.join(path,file))

#feature_cols = ['AirTemp_degC', 'Longwave_Wm-2', 'Latent_Wm-2', 'Sensible_Wm-2', 'Shortwave_Wm-2',
#                'lightExtinct_m-1', 'ShearStress_Nm-2',
#                 'day_of_year', 'time_of_day', 'ice', 'snow', 'snowice','Volume_m2','Osgood','MaxDepth_m',
#                'MeanDepth_m','Area_m2'] + incoming_temp

feature_cols = ['AirTemp_degC', 'Longwave_Wm-2', 'Latent_Wm-2', 'Sensible_Wm-2', 'Shortwave_Wm-2',
                'lightExtinct_m-1', 'ShearStress_Nm-2',
                 'day_of_year', 'time_of_day', 'ice', 'snow', 'snowice'] + incoming_temp

date_col = ['time']

target_cols = outgoing_temp

In [10]:
#dx = pd.read_csv(os.path.join(path, metadata))

# Extract all col names from Metadata
#feature_cols = dx[dx['column_type']=='feature']['column_names'].tolist()  # feature colums represent the input drivers
#target_cols = dx[dx['column_type']=='target']['column_names'].tolist()   # target column represent the chlorophyll values
#date_col = dx[dx['column_type']=='date']['column_names'].tolist()[0]    # date column stores the date timeline

In [11]:
# Specify whether we want to add chlorophyll to the input feature set
#feature_cols += target_cols
feature_cols

['AirTemp_degC',
 'Longwave_Wm-2',
 'Latent_Wm-2',
 'Sensible_Wm-2',
 'Shortwave_Wm-2',
 'lightExtinct_m-1',
 'ShearStress_Nm-2',
 'day_of_year',
 'time_of_day',
 'ice',
 'snow',
 'snowice',
 'temp_diff03_0.5',
 'temp_diff03_1.0',
 'temp_diff03_1.5',
 'temp_diff03_2.0',
 'temp_diff03_2.5',
 'temp_diff03_3.0',
 'temp_diff03_3.5',
 'temp_diff03_4.0',
 'temp_diff03_4.5',
 'temp_diff03_5.0',
 'temp_diff03_5.5',
 'temp_diff03_6.0',
 'temp_diff03_6.5',
 'temp_diff03_7.0',
 'temp_diff03_7.5',
 'temp_diff03_8.0',
 'temp_diff03_8.5',
 'temp_diff03_9.0',
 'temp_diff03_9.5',
 'temp_diff03_10.0',
 'temp_diff03_10.5',
 'temp_diff03_11.0',
 'temp_diff03_11.5',
 'temp_diff03_12.0',
 'temp_diff03_12.5',
 'temp_diff03_13.0',
 'temp_diff03_13.5',
 'temp_diff03_14.0',
 'temp_diff03_14.5',
 'temp_diff03_15.0',
 'temp_diff03_15.5',
 'temp_diff03_16.0',
 'temp_diff03_16.5',
 'temp_diff03_17.0',
 'temp_diff03_17.5',
 'temp_diff03_18.0',
 'temp_diff03_18.5',
 'temp_diff03_19.0',
 'temp_diff03_19.5',
 'temp_di

In [12]:
target_cols

['temp_conv04_0.5',
 'temp_conv04_1.0',
 'temp_conv04_1.5',
 'temp_conv04_2.0',
 'temp_conv04_2.5',
 'temp_conv04_3.0',
 'temp_conv04_3.5',
 'temp_conv04_4.0',
 'temp_conv04_4.5',
 'temp_conv04_5.0',
 'temp_conv04_5.5',
 'temp_conv04_6.0',
 'temp_conv04_6.5',
 'temp_conv04_7.0',
 'temp_conv04_7.5',
 'temp_conv04_8.0',
 'temp_conv04_8.5',
 'temp_conv04_9.0',
 'temp_conv04_9.5',
 'temp_conv04_10.0',
 'temp_conv04_10.5',
 'temp_conv04_11.0',
 'temp_conv04_11.5',
 'temp_conv04_12.0',
 'temp_conv04_12.5',
 'temp_conv04_13.0',
 'temp_conv04_13.5',
 'temp_conv04_14.0',
 'temp_conv04_14.5',
 'temp_conv04_15.0',
 'temp_conv04_15.5',
 'temp_conv04_16.0',
 'temp_conv04_16.5',
 'temp_conv04_17.0',
 'temp_conv04_17.5',
 'temp_conv04_18.0',
 'temp_conv04_18.5',
 'temp_conv04_19.0',
 'temp_conv04_19.5',
 'temp_conv04_20.0',
 'temp_conv04_20.5',
 'temp_conv04_21.0',
 'temp_conv04_21.5',
 'temp_conv04_22.0',
 'temp_conv04_22.5',
 'temp_conv04_23.0',
 'temp_conv04_23.5',
 'temp_conv04_24.0',
 'temp_conv

#### Create an utility object
An object of the Utils class, it contains all the utility functions like splitting train and test data, normalizing the data, etc.

In [13]:
'''
Utility instance - to perform data processing, train test split
'''
utils = Utils(num_features=len(feature_cols), inp_cols=feature_cols, target_cols=target_cols, date_col=date_col,
              input_window=input_window, output_window=output_window, num_out_features=output_size, stride=stride)

#### Read the dataset

In [14]:
'''
Read data
'''
df = pd.read_csv(path+file)

#### Train Test split
Ideally, a 3-way split is done - train, val and test. The validation split is generally used to tune the hyper-parameters during training. Once the hyper-parameters are tuned, the model
is re-trained on the train+val data. To keep the notebook short and simple, hyper-parameter tuning is not included

In [15]:
'''
Split data into train and test
'''
df_train, df_test = utils.train_test_split(df, split_ratio=split_ratio)

In [16]:
def cycle_encode(x, period):
    sin = np.sin(2*math.pi*x/period)
    cos = np.cos(2*math.pi*x/period)
    
    return sin, cos

In [17]:
train_doy_sin, train_doy_cos = cycle_encode(df_train.day_of_year.values, 365)

In [18]:
train_tod_sin, train_tod_cos = cycle_encode(df_train.time_of_day.values, 24)

In [19]:
test_doy_sin, test_doy_cos = cycle_encode(df_test.day_of_year.values, 365)

In [20]:
test_tod_sin, test_tod_cos = cycle_encode(df_test.time_of_day.values, 24)

In [21]:
feature_cols.remove('day_of_year')
feature_cols.remove('time_of_day')
feature_cols

['AirTemp_degC',
 'Longwave_Wm-2',
 'Latent_Wm-2',
 'Sensible_Wm-2',
 'Shortwave_Wm-2',
 'lightExtinct_m-1',
 'ShearStress_Nm-2',
 'ice',
 'snow',
 'snowice',
 'temp_diff03_0.5',
 'temp_diff03_1.0',
 'temp_diff03_1.5',
 'temp_diff03_2.0',
 'temp_diff03_2.5',
 'temp_diff03_3.0',
 'temp_diff03_3.5',
 'temp_diff03_4.0',
 'temp_diff03_4.5',
 'temp_diff03_5.0',
 'temp_diff03_5.5',
 'temp_diff03_6.0',
 'temp_diff03_6.5',
 'temp_diff03_7.0',
 'temp_diff03_7.5',
 'temp_diff03_8.0',
 'temp_diff03_8.5',
 'temp_diff03_9.0',
 'temp_diff03_9.5',
 'temp_diff03_10.0',
 'temp_diff03_10.5',
 'temp_diff03_11.0',
 'temp_diff03_11.5',
 'temp_diff03_12.0',
 'temp_diff03_12.5',
 'temp_diff03_13.0',
 'temp_diff03_13.5',
 'temp_diff03_14.0',
 'temp_diff03_14.5',
 'temp_diff03_15.0',
 'temp_diff03_15.5',
 'temp_diff03_16.0',
 'temp_diff03_16.5',
 'temp_diff03_17.0',
 'temp_diff03_17.5',
 'temp_diff03_18.0',
 'temp_diff03_18.5',
 'temp_diff03_19.0',
 'temp_diff03_19.5',
 'temp_diff03_20.0',
 'temp_diff03_20.5',

#### Normalize the data
Standard normalization - 0 mean and 1 standard deviation

In [22]:
utils.inp_cols

['AirTemp_degC',
 'Longwave_Wm-2',
 'Latent_Wm-2',
 'Sensible_Wm-2',
 'Shortwave_Wm-2',
 'lightExtinct_m-1',
 'ShearStress_Nm-2',
 'ice',
 'snow',
 'snowice',
 'temp_diff03_0.5',
 'temp_diff03_1.0',
 'temp_diff03_1.5',
 'temp_diff03_2.0',
 'temp_diff03_2.5',
 'temp_diff03_3.0',
 'temp_diff03_3.5',
 'temp_diff03_4.0',
 'temp_diff03_4.5',
 'temp_diff03_5.0',
 'temp_diff03_5.5',
 'temp_diff03_6.0',
 'temp_diff03_6.5',
 'temp_diff03_7.0',
 'temp_diff03_7.5',
 'temp_diff03_8.0',
 'temp_diff03_8.5',
 'temp_diff03_9.0',
 'temp_diff03_9.5',
 'temp_diff03_10.0',
 'temp_diff03_10.5',
 'temp_diff03_11.0',
 'temp_diff03_11.5',
 'temp_diff03_12.0',
 'temp_diff03_12.5',
 'temp_diff03_13.0',
 'temp_diff03_13.5',
 'temp_diff03_14.0',
 'temp_diff03_14.5',
 'temp_diff03_15.0',
 'temp_diff03_15.5',
 'temp_diff03_16.0',
 'temp_diff03_16.5',
 'temp_diff03_17.0',
 'temp_diff03_17.5',
 'temp_diff03_18.0',
 'temp_diff03_18.5',
 'temp_diff03_19.0',
 'temp_diff03_19.5',
 'temp_diff03_20.0',
 'temp_diff03_20.5',

In [23]:
'''
Data Scaling
'''
df_train = utils.normalize(df_train)

In [24]:
df_test = utils.normalize(df_test, use_stat=True)

In [25]:
feature_cols += ['doy_sin', 'doy_cos', 'tod_sin', 'tod_cos']

In [26]:
utils.inp_cols

['AirTemp_degC',
 'Longwave_Wm-2',
 'Latent_Wm-2',
 'Sensible_Wm-2',
 'Shortwave_Wm-2',
 'lightExtinct_m-1',
 'ShearStress_Nm-2',
 'ice',
 'snow',
 'snowice',
 'temp_diff03_0.5',
 'temp_diff03_1.0',
 'temp_diff03_1.5',
 'temp_diff03_2.0',
 'temp_diff03_2.5',
 'temp_diff03_3.0',
 'temp_diff03_3.5',
 'temp_diff03_4.0',
 'temp_diff03_4.5',
 'temp_diff03_5.0',
 'temp_diff03_5.5',
 'temp_diff03_6.0',
 'temp_diff03_6.5',
 'temp_diff03_7.0',
 'temp_diff03_7.5',
 'temp_diff03_8.0',
 'temp_diff03_8.5',
 'temp_diff03_9.0',
 'temp_diff03_9.5',
 'temp_diff03_10.0',
 'temp_diff03_10.5',
 'temp_diff03_11.0',
 'temp_diff03_11.5',
 'temp_diff03_12.0',
 'temp_diff03_12.5',
 'temp_diff03_13.0',
 'temp_diff03_13.5',
 'temp_diff03_14.0',
 'temp_diff03_14.5',
 'temp_diff03_15.0',
 'temp_diff03_15.5',
 'temp_diff03_16.0',
 'temp_diff03_16.5',
 'temp_diff03_17.0',
 'temp_diff03_17.5',
 'temp_diff03_18.0',
 'temp_diff03_18.5',
 'temp_diff03_19.0',
 'temp_diff03_19.5',
 'temp_diff03_20.0',
 'temp_diff03_20.5',

In [27]:
df_train['doy_sin'] = train_doy_sin
df_train['doy_cos'] = train_doy_cos
df_test['doy_sin'] = test_doy_sin
df_test['doy_cos'] = test_doy_cos

In [28]:
df_train['tod_sin'] = train_tod_sin
df_train['tod_cos'] = train_tod_cos
df_test['tod_sin'] = test_tod_sin
df_test['tod_cos'] = test_tod_cos

In [29]:
'''
convert the mean and std to torch
'''
utils.y_mean = torch.tensor(utils.y_mean, device=device)
utils.y_std = torch.tensor(utils.y_std, device=device)

In [30]:
utils.y_mean

tensor([10.1005, 10.1581, 10.2029, 10.2337, 10.2534, 10.2615, 10.2585, 10.2492,
        10.2300, 10.2000, 10.1660, 10.1176, 10.0438,  9.9460,  9.8580,  9.7349,
         9.6189,  9.4899,  9.3367,  9.1905,  9.0190,  8.8053,  8.5458,  8.2925,
         8.0828,  7.8673,  7.6295,  7.4294,  7.2482,  7.0237,  6.7800,  6.5822,
         6.4095,  6.2334,  6.0480,  5.8627,  5.7069,  5.6065,  5.5390,  5.5035,
         5.4891,  5.4737,  5.4823,  5.5070,  5.5376,  5.5717,  5.6058,  5.6386,
         5.6477,  5.6179], device='cuda:0', dtype=torch.float64)

In [31]:
utils.y_std

tensor([9.3025, 9.2315, 9.1628, 9.1012, 9.0451, 8.9889, 8.9332, 8.8808, 8.8272,
        8.7672, 8.7094, 8.6384, 8.5307, 8.4011, 8.2920, 8.1573, 8.0399, 7.8841,
        7.7062, 7.5438, 7.3588, 7.1610, 6.9432, 6.7433, 6.5725, 6.3599, 6.0940,
        5.8810, 5.6637, 5.3618, 5.0456, 4.8153, 4.6246, 4.4222, 4.1840, 3.9354,
        3.7430, 3.6266, 3.5348, 3.4670, 3.3968, 3.2924, 3.2293, 3.1870, 3.1521,
        3.1201, 3.0900, 3.0593, 3.0284, 2.8981], device='cuda:0',
       dtype=torch.float64)

In [32]:
utils.num_features = len(utils.inp_cols)

#### Create train and test samples
Each sample is created using a sliding window. 1 sliding window = 1 lookback window + 1 horizon window = 1 sample

In [33]:
'''
Prepare data : 1 training sample = lookback window + horizon window
'''
Xtrain, Ytrain = utils.windowed_dataset(df_train)
Xtest, Ytest = utils.windowed_dataset(df_test)

In [34]:
Xtrain.shape

(31399, 72, 64)

In [35]:
Ytrain.shape

(31399, 72, 50)

In [36]:
Xtest.shape

(20885, 72, 64)

In [37]:
Ytest.shape

(20885, 72, 50)

#### Datatype conversion to torch

In [38]:
'''
Convert data into torch type
'''
X_train, Y_train, X_test, Y_test = utils.numpy_to_torch(Xtrain, Ytrain, Xtest, Ytest)

## 3. Modeling

#### Define the model

In [39]:
'''
Create the seq2seq model
'''
model = seq2seq(input_size = X_train.shape[2], 
                hidden_size = hidden_feature_size, 
                output_size=output_size,
                model_type=model_type,
                num_layers = num_layers,
                utils=utils,
                dropout=dropout,
                device=device
               )

#### Train the model

In [ ]:
'''
Train the model
'''
config = {
    "batch_size": batch_size,
    "epochs": epochs,
    "learning_rate": learning_rate,
    "eval_freq": eval_freq,
    "batch_shuffle": batch_shuffle,
    "dropout":dropout,
    "num_layers": num_layers,
    "hidden_feature_size": hidden_feature_size,
    "model_type": model_type,
    "teacher_forcing_ratio": teacher_forcing_ratio,
    "max_lr": max_lr,
    "div_factor": div_factor,
    "pct_start": pct_start,
    "anneal_strategy": anneal_strategy,
    "final_div_factor": final_div_factor,
    "dataset": file,
    "split_ratio":split_ratio,
    "input_window":input_window,
    "output_window":output_window,
    "early_stop_thres":thres,
    "early_stop_delta":delta,
    "early_stop":early_stop,
    "weight_decay":weight_decay
}
loss, test_rmse, train_rmse = model.train_model(X_train, 
                                                Y_train,
                                                X_test,
                                                Y_test,
                                                target_len = output_window,
                                                config = config,
                                                training_prediction = training_prediction,  
                                                dynamic_tf = False,
                                                project_name = wandb_project,
                                                run_name = wandb_run,
                                                save_code = save_code)

  7%|█████▎                                                                           | 64/982 [00:51<05:53,  2.59it/s]


  8%|█████▋                                                                   | 76/982 [1:24:30<378:35:11, 1504.32s/it]

In [ ]:
plt.figure(figsize=(5,4), dpi=150)
plt.plot(train_rmse, lw=2.0, label='train_rmse')
plt.plot(test_rmse, lw=2.0, label='test_rmse')
plt.yscale("log")
plt.grid("on", alpha=0.2)
plt.legend()
plt.show()

#### Plot the train test curves

In [ ]:
plt.figure(figsize=(5,4), dpi=150)
plt.plot(train_rmse, lw=2.0, label='train_rmse')
plt.plot(test_rmse, lw=2.0, label='test_rmse')
plt.yscale("log")
plt.grid("on", alpha=0.2)
plt.legend()
plt.show()

#### Save the model

In [ ]:
load = False

# If load=True, specify the model to load in the below line
MODEL_PATH = "./models/model_weights_test_run_2023-03-21_22:42:45.577525"

In [ ]:

if load:
    model.load_state_dict(torch.load(MODEL_PATH))
else:
    MODEL_PATH='./models/model_weights_{}'.format(wandb_run[:22])
    if not os.path.exists('./models'):
        os.mkdir('./models')
    torch.save(model.state_dict(), MODEL_PATH)

In [ ]:
'''
Perform evaluation
'''
train_eval_dict = model.evaluate_batch(X_train.to(device), Y_train.to(device))
test_eval_dict = model.evaluate_batch(X_test.to(device), Y_test.to(device))

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
Y_test.shape

In [ ]:
test_eval_dict['y_true'][1001][2]

In [ ]:
test_eval_dict['y_pred'][1001][2]

## 4. Plotting and Evaluation

In [ ]:
'''
Create plot tables for T+n th predictions
'''
train_gt = train_eval_dict['y_true']
train_gt_df = pd.DataFrame(train_gt.cpu().numpy()[:,:,0])
train_gt_values = np.append(train_gt_df[0].values, train_gt_df.iloc[-1,1:]) # ground-truth values for train data

test_gt = test_eval_dict['y_true']
test_gt_df = pd.DataFrame(test_gt.cpu().numpy()[:,:,0])
test_gt_values = np.append(test_gt_df[0].values, test_gt_df.iloc[-1,1:]) # ground-truth values for test data

train_pred = train_eval_dict['y_pred'] # model predicted values for train data
test_pred = test_eval_dict['y_pred'] # model predicted values for test data

df_train_comp = df_train
#df_train_comp=df_train_comp.rename(columns = {'time':'Date'})
#print(df_train_comp.Date)

df_test_comp = df_test
#df_test_comp=df_test_comp.rename(columns = {'time':'Date'})
#print(df_test_comp.Date)

print(df_train_comp.shape)
print(train_pred.shape)
print(train_gt_values.shape)

train_T_pred_table, train_plot_df, plot_train_gt_values = utils.predictionTable(df_train_comp, train_pred, train_gt_values)

test_T_pred_table, test_plot_df, plot_test_gt_values = utils.predictionTable(df_test_comp, test_pred, test_gt_values)

In [ ]:
'''
Generate the plots on train data
'''

# Specify the list of T+n predictions to plot
horizon_range = [1,7, 14] # this will plot T+1 and T+n predictions w.r.t Ground truth

utils.plotTable(train_plot_df, plot_train_gt_values, horizon_range)

In [ ]:
'''
Generate the plots on test data
'''

# Specify the list of T+n predictions to plot
horizon_range = [1,7,14] # this will plot T+1 and T+n predictions w.r.t Ground truth

utils.plotTable(test_plot_df, plot_test_gt_values, horizon_range)

#### Compute the RMSE values

In [ ]:
'''
Compute train rmse

- Train RMSE values for all T+n th predictions. The index represents the T+n

'''
rmse_values = []
for i in range(output_window):
    rmse_values.append(utils.compute_rmse(i, train_T_pred_table, train_gt_values))
rmse_values = pd.DataFrame(rmse_values, columns=['RMSE'], index=range(1,output_window+1))
rmse_values

In [ ]:
'''
Compute test rmse

- Test RMSE values for all T+n th predictions. The index represents the T+n

'''
test_rmse_values = []
for i in range(output_window):
    test_rmse_values.append(utils.compute_rmse(i, test_T_pred_table, test_gt_values))
test_rmse_values = pd.DataFrame(test_rmse_values, columns=['RMSE'], index=range(1,output_window+1))
test_rmse_values